In [1]:
import tensorflow as tf
import glob
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import Input, LeakyReLU, Conv2D as conv2D, Dropout, Add, UpSampling2D, concatenate
import cv2
import random
import os
import PIL
from PIL import Image, ImageOps
from matplotlib import colors, cm
from ImprovedUnet import improved_unet


HEIGHT = 256
WIDTH = 256
TRAIN_SIZE = 1815
TEST_SIZE = 389
VALI_SIZE = 389

In [2]:
# Given prediction images and ground truth imgaes. Return the dice similarity coefficient
def dice_coef(predictions, truth, axis=(1,2,3)):
    predictions = tf.convert_to_tensor(predictions, np.float32)
    truth = tf.convert_to_tensor(truth, np.float32)
    numerator = (2.0 * (tf.reduce_sum(predictions * truth, axis=axis))) + 1
    denominator = tf.reduce_sum(predictions, axis=axis) + tf.reduce_sum(truth, axis=axis) + 1
    coef = tf.reduce_mean(numerator / denominator)
    return coef

# Return the dice similarity coefficient loss
def dice_coef_loss(predictions, truth):
    return 1 - dice_coef(predictions, truth)

# Read images from row image file name and ground truth file name
def read_images(image_file, mask_file):
    img = cv2.imread(image_file)
    img = cv2.resize(img, (HEIGHT, WIDTH))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img/255
    img = (img - img.mean())/img.std()
    mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (HEIGHT, WIDTH))
    mask = np.round(mask/255)
    mask = mask[:, :, np.newaxis]
    return img, mask

# Image gererator loaded batchsize number of random pairs of row images and ground truth images.
def image_generator(image_fnames, mask_fnames, batchsize):
    count = len(image_fnames)
    while True:
        random_indices = random.sample(range(count), batchsize)
        batch_img, batch_mask = [], []
        for index in random_indices:
            img, mask = read_images(image_fnames[index], mask_fnames[index])
            batch_img.append(img)
            batch_mask.append(mask)
        images = np.stack(batch_img, axis=0)
        masks = np.stack(batch_mask, axis=0)
        yield images, masks 

# Plot five pair of row images and ground truth images from data generator.
def plot_gen(data_gen):
    images, masks = next(data_gen)
    plot_images(images, 3)
    plot_images(masks, 1)

# Plot five images in one row. Set dim=1 when plotting gray images, set dim to 3 when plotting color images.
def plot_images(images, dim):
    fig, axs = plt.subplots(1, 5, sharey=True, figsize=(15,15))
    for i in range(5):
        if dim == 3:
            img = images[i, :, :, :]
            norm = colors.LogNorm(img.mean() + 0.5 * img.std(), img.max(), clip='True')  
            axs[i].imshow(img, cmap=cm.gray, norm=norm, origin="lower")
        elif dim == 1:
            img = images[i, :, :, 0]
            axs[i].imshow(img, cmap='gray')
    plt.show()
    
# Plot five pair of images for training, testing and validation generator to ensure data is correctly loaded
def check_loaded(train_gen, test_gen, val_gen):
    plot_gen(train_gen)
    plot_gen(test_gen)
    plot_gen(val_gen)

# Plot five pairs of row images, ground truth images and their corresponding predictions.
def plot_comparison(images, test_masks, predictions):
    images, test_masks, predictions = generate_random_images(images, test_masks, predictions)
    plot_images(images, 3)
    plot_images(test_masks, 1)    
    plot_images(predictions, 1)

# Generate five random images for row images, ground truth images and their corresponding predictions.
def generate_random_images(test_images, test_masks, predictions):
    random_indices = random.sample(range(TEST_SIZE), 5)
    print(random_indices)
    imgs = []
    preds = []
    masks = []
    for i in random_indices:
        imgs.append(test_images[i, :, :, :])
        masks.append(test_masks[i, :, :, :])
        preds.append(predictions[i, :, :, :])
    imgs = np.stack(imgs, axis=0)
    masks = np.stack(masks, axis=0)
    preds = np.stack(preds, axis=0)
    return imgs, masks, preds

# Load images from raw images and ground truth file names, return them as numpy arrays
def load_images_from_fn(image_fnames, mask_fnames):
    images = []
    masks = []
    for i in range(len(image_fnames)):
        img, mask = read_images(image_fnames[i], mask_fnames[i])
        images.append(img)
        masks.append(mask)
    images = np.stack(images, axis=0)
    masks = np.stack(masks, axis=0)
    return images, masks

# Load raw images using PIL
def load_original_test_images(test_x):
    images = []
    for file in test_x:
        img = PIL.Image.open(str(file))
        img = ImageOps.flip(img)
        img = img.resize((HEIGHT, WIDTH))
        img = np.array(img.getdata()).reshape(HEIGHT, WIDTH, 3)
        images.append(img)
    images = np.stack(images, axis=0)
    return images

In [3]:
# Load images, return training, validation generator, return test_masks, test_images in numpy format.
# return test_images_forplot because the test_images loaded are normalized. 
def load_data():
    input_images = sorted(glob.glob("H:\\COMP3710/report/ISIC2018_Task1-2_Training_Data/input/*.jpg"))
    output_masks = sorted(glob.glob("H:\\COMP3710/report/ISIC2018_Task1-2_Training_Data/truth/*.png"))
    
    # Separate file names to train, validation and test.
    DATASET_SIZE = len(input_images)
    train_size = int(DATASET_SIZE * 0.7)
    val_size = test_size = int(DATASET_SIZE * 0.15)
    train_x = input_images[0:train_size]
    train_y = output_masks[0:train_size]
    test_x = input_images[train_size:train_size+test_size]
    test_y = output_masks[train_size:train_size+test_size]
    val_x = input_images[train_size+test_size:]
    val_y = output_masks[train_size+test_size:]
    
    train_gen = image_generator(train_x, train_y, 10)
    val_gen = image_generator(val_x, val_y, 10)
    test_images, test_masks = load_images_from_fn(test_x, test_y)
    test_images_forplot = load_original_test_images(test_x)
    print('Loading success')
    return train_gen, val_gen, test_masks, test_images, test_images_forplot


def compile_model(train_gen, val_gen):
    model = improved_unet(HEIGHT, WIDTH)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[dice_coef, 'acc'])
    # Create check point
    checkpoint_filepath = '/tmp/checkpoint'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='acc',
        mode='max',
        save_best_only=True)
    # Compile model
    train_history = model.fit(train_gen, epochs=250, steps_per_epoch=10, validation_data = val_gen, validation_steps=10)
    return model, train_history

def plot_training_results(train_history):
    plt.plot(train_history.history['loss'], 'k', label='Training loss')
    plt.plot(train_history.history['val_loss'], 'b', label = 'Validation loss')
    plt.title('Loss of trained data')
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()

    plt.figure()
    plt.plot(train_history.history['acc'], 'k', label='Training accuracy')
    plt.plot(train_history.history['val_acc'], 'b', label='Validation accuracy')
    plt.title('Accuracy of trained data')
    plt.xlabel('Iterations')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid()

    plt.figure()
    plt.plot(train_history.history['dice_coef'], 'k', label='Training dsc ')
    plt.plot(train_history.history['val_dice_coef'], 'b', label='Validation dsc')
    plt.title('Dice similarity coefficient of trained data')
    plt.xlabel('Iterations')
    plt.ylabel('Dice similarity coefficient')
    plt.legend()
    plt.grid()


In [ ]:
def main():
    train_gen, val_gen, test_masks, test_images, test_images_forplot = load_data()
    model, train_history = compile_model(train_gen, val_gen)
    plot_training_results(train_history)
    predictions = np.round(model.predict(test_images))
    plot_comparison(test_images_forplot, test_masks, predictions)
    dsc = dice_coef(predictions, test_masks)
    dsc = dsc.numpy()
    print('Dice similarity coefficient: ' + str(dsc))

if __name__ == "__main__":
    main()

Loading success
Tensor("input_1:0", shape=(None, 256, 256, 3), dtype=float32)
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 10 steps, validate for 10 steps
Epoch 1/250
10/10 [==============================] - 16s 2s/step - loss: 0.5738 - dice_coef: 0.2378 - acc: 0.7741 - val_loss: 0.4798 - val_dice_coef: 0.3071 - val_acc: 0.8865
Epoch 2/250
10/10 [==============================] - 14s 1s/step - loss: 0.4526 - dice_coef: 0.2825 - acc: 0.8373 - val_loss: 0.4631 - val_dice_coef: 0.3910 - val_acc: 0.8549
Epoch 3/250
10/10 [==============================] - 16s 2s/step - loss: 0.3919 - dice_coef: 0.3401 - acc: 0.8749 - val_loss: 0.3936 - val_dice_coef: 0.3799 - val_acc: 0.8832
Epoch 4/250
10/10 [==============================] - 14s 1s/step - loss: 0.4108 - dice_coef: 0.2963 - acc: 0.8740 - val_loss: 0.4174 - val_dice_coef: 0.3786 - val_acc: 0.8654
Epoch 5/250
10/10 [==============================] - 12s 1s/step - loss: 0.4826 - dice_coef: 0.2988 - acc: 0.8354 - val_loss: 0.45

Epoch 46/250
10/10 [==============================] - 14s 1s/step - loss: 0.1699 - dice_coef: 0.7019 - acc: 0.9352 - val_loss: 0.2154 - val_dice_coef: 0.6964 - val_acc: 0.9174
Epoch 47/250
10/10 [==============================] - 14s 1s/step - loss: 0.2156 - dice_coef: 0.6545 - acc: 0.9145 - val_loss: 0.2562 - val_dice_coef: 0.6790 - val_acc: 0.9092
Epoch 48/250
10/10 [==============================] - 15s 1s/step - loss: 0.1632 - dice_coef: 0.6213 - acc: 0.9429 - val_loss: 0.1933 - val_dice_coef: 0.7392 - val_acc: 0.9207
Epoch 49/250
10/10 [==============================] - 14s 1s/step - loss: 0.1821 - dice_coef: 0.7167 - acc: 0.9252 - val_loss: 0.2095 - val_dice_coef: 0.6918 - val_acc: 0.9269
Epoch 50/250
10/10 [==============================] - 13s 1s/step - loss: 0.2314 - dice_coef: 0.5958 - acc: 0.9133 - val_loss: 0.2223 - val_dice_coef: 0.6228 - val_acc: 0.9161
Epoch 51/250
10/10 [==============================] - 15s 1s/step - loss: 0.2146 - dice_coef: 0.6220 - acc: 0.9217 - val

10/10 [==============================] - 21s 2s/step - loss: 0.2172 - dice_coef: 0.6438 - acc: 0.9074 - val_loss: 0.2430 - val_dice_coef: 0.6682 - val_acc: 0.9037
Epoch 93/250
10/10 [==============================] - 21s 2s/step - loss: 0.2129 - dice_coef: 0.6731 - acc: 0.9300 - val_loss: 0.2493 - val_dice_coef: 0.6577 - val_acc: 0.9092
Epoch 94/250
10/10 [==============================] - 14s 1s/step - loss: 0.2056 - dice_coef: 0.6138 - acc: 0.9209 - val_loss: 0.2089 - val_dice_coef: 0.7337 - val_acc: 0.9218
Epoch 95/250
10/10 [==============================] - 20s 2s/step - loss: 0.1903 - dice_coef: 0.6657 - acc: 0.9261 - val_loss: 0.2089 - val_dice_coef: 0.7147 - val_acc: 0.9268
Epoch 96/250
10/10 [==============================] - 15s 1s/step - loss: 0.1456 - dice_coef: 0.6423 - acc: 0.9450 - val_loss: 0.1868 - val_dice_coef: 0.7672 - val_acc: 0.9272
Epoch 97/250
10/10 [==============================] - 16s 2s/step - loss: 0.1401 - dice_coef: 0.7355 - acc: 0.9454 - val_loss: 0.2226

10/10 [==============================] - 16s 2s/step - loss: 0.1664 - dice_coef: 0.7115 - acc: 0.9323 - val_loss: 0.1966 - val_dice_coef: 0.7243 - val_acc: 0.9217
Epoch 139/250
10/10 [==============================] - 14s 1s/step - loss: 0.1530 - dice_coef: 0.7235 - acc: 0.9369 - val_loss: 0.1844 - val_dice_coef: 0.7678 - val_acc: 0.9301
Epoch 140/250
10/10 [==============================] - 21s 2s/step - loss: 0.1553 - dice_coef: 0.7407 - acc: 0.9366 - val_loss: 0.1820 - val_dice_coef: 0.7115 - val_acc: 0.9342
Epoch 141/250
10/10 [==============================] - 18s 2s/step - loss: 0.1211 - dice_coef: 0.6947 - acc: 0.9537 - val_loss: 0.1887 - val_dice_coef: 0.7535 - val_acc: 0.9212
Epoch 142/250
10/10 [==============================] - 19s 2s/step - loss: 0.1305 - dice_coef: 0.6904 - acc: 0.9494 - val_loss: 0.2280 - val_dice_coef: 0.7202 - val_acc: 0.9151
Epoch 143/250
10/10 [==============================] - 21s 2s/step - loss: 0.1818 - dice_coef: 0.7081 - acc: 0.9300 - val_loss: 0

10/10 [==============================] - 15s 2s/step - loss: 0.1198 - dice_coef: 0.7635 - acc: 0.9521 - val_loss: 0.1354 - val_dice_coef: 0.7740 - val_acc: 0.9475
Epoch 185/250
10/10 [==============================] - 13s 1s/step - loss: 0.1407 - dice_coef: 0.7583 - acc: 0.9453 - val_loss: 0.1940 - val_dice_coef: 0.6992 - val_acc: 0.9266
Epoch 186/250
10/10 [==============================] - 14s 1s/step - loss: 0.1314 - dice_coef: 0.7508 - acc: 0.9469 - val_loss: 0.1662 - val_dice_coef: 0.7755 - val_acc: 0.9337
Epoch 187/250
10/10 [==============================] - 15s 2s/step - loss: 0.1147 - dice_coef: 0.7624 - acc: 0.9514 - val_loss: 0.2171 - val_dice_coef: 0.7735 - val_acc: 0.9146
Epoch 188/250
10/10 [==============================] - 14s 1s/step - loss: 0.1437 - dice_coef: 0.7439 - acc: 0.9446 - val_loss: 0.1935 - val_dice_coef: 0.7411 - val_acc: 0.9235
Epoch 189/250
10/10 [==============================] - 12s 1s/step - loss: 0.1612 - dice_coef: 0.7330 - acc: 0.9439 - val_loss: 0